# read in wet files to spark

extract the url the article, map in to url->text pair

In [ ]:
def censor(text):
    censoredText = ""
    URI = ""
    for line in text.split('\n'):
        if line.startswith("WARC-Target-URI:"):
            URI = line[17:]
        elif line.startswith("WARC") or line.startswith("Content-"):
            continue
        else:
            censoredText += line
    return (URI,censoredText)

delete non-text pair

In [64]:
def is_text(markedtext):
    if markedtext[0]:
        return True
    else:
        return False

sepertate textfile by WARC/1.0

In [65]:
from pyspark.sql.types import StructType, StructField, DoubleType, StringType
schema = StructType([
    StructField("URI", StringType(), True), StructField("text", StringType(), False)
])

def load_article(category_name):
    text_file = sc.newAPIHadoopFile("small.wet", "org.apache.hadoop.mapreduce.lib.input.TextInputFormat",
            "org.apache.hadoop.io.LongWritable", "org.apache.hadoop.io.Text",
            conf={"textinputformat.record.delimiter": "WARC/1.0"}).map(lambda l:l[1])
    return text_file.map(censor).filter(is_text).toDF(schema)

In [68]:
micro_articles = load_article("small.wet")

In [67]:
micro_articles.show()

+--------------------+--------------------+
|                 URI|                text|
+--------------------+--------------------+
|http://01marocain...|01marocainLe site...|
|http://10.asiae.c...|�ƽþư��� �ƽþư��� �...|
|http://10000birds...|10,000 Birds | Wh...|
|http://1012lounge...|10/12 LoungeHomeE...|
|http://1015store....|DRESSES BY COLORS...|
|http://1019amprad...|Happy Holidays « ...|
|http://102therive...|Get Important New...|
|http://1035kissfm...|'Reading Rainbow'...|
|http://1037theq.i...|Tiesto celebrates...|
|http://1065ctq.ih...|New Electronic Li...|
|http://10kbullets...|Neon Nights – 10,...|
|http://12160.info...|Both Republicans ...|
|http://12under12u...|12under12under201...|
|http://1350kman.c...|Settlement Reache...|
|http://14ers.com/...|14ers.com • Login...|
|http://16sarkisoz...|Şarkı sözü: Nasip...|
|http://17hmr.net/...|An Error Has Occu...|
|http://17hmr.net/...|LoginWelcome, Gue...|
|http://17hmr.net/...|An Error Has Occu...|
|http://17hmr.net/...|An Error H